<a href="https://colab.research.google.com/github/shubham-arote/Multiuser-ReAct-agent/blob/main/Building_multiuser_ReAct_agent_chatbot_with_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building a multi-user ReAct AI  Agent Chatbot with  LangGraph


In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langgraph

In [ ]:
!pip install langchain-google-genai

In [ ]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['WEATHER_API_KEY'] = userdata.get('WEATHER_API_KEY')
os.environ['GOOGLE_API_KEY']=userdata.get('GEMINI_API_KEY')
os.environ['TAVILY_API_KEY']=userdata.get('TAVILY_API_KEY')

In [ ]:
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

In [ ]:
WEATHER_API_KEY = os.environ['WEATHER_API_KEY']

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import json
import requests

tv_search = TavilySearchResults(max_results=3, search_depth='advance', max_tokens=10000)

@tool
def search_web(query:str) -> list:
    """Search the web for the given query"""
    search_tool = TavilySearchResults(max_results=2)
    results = search_tool.invoke(query)
    return results

@tool
def get_weather(query: str) -> list:
    """Search weather api to get current results"""
    base_url="http://api.weatherapi.com/v1/current.json"
    complete_url= f"{base_url}?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not found"
tools = [search_web, get_weather]

In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
chat_gpt = ChatGroq(
    model = "llama3-8b-8192",
    temperature = 0
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

Build and test ai agent with langgraph

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
SYS_PROMPT = """Act as a helpful assistant.
                Use the tools at your disposal to perform tasks as needed
                 -get_weather: whenever user asks get the weather of a place
                 -search_web : whenever user asks for information on current events or
                 if you don't know the answer """

prompt_template = ChatPromptTemplate.from_messages([
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="messages"),
])

def add_sys_prompt(messages):
    return prompt_template.invoke({"messages":messages}).to_messages()

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools=tools, messages_modifier=add_sys_prompt)

In [ ]:
query = """tell me who won the champions league in 2024
        show some detailed information about the match also"""

response = agent.invoke({"messages":['user', query]})
from IPython.display import Markdown, display
display(Markdown(response['messages'][-1].content))

Real Madrid won the Champions League in 2024 after defeating Borussia Dortmund in the final. 

Goals from Dani Carvajal and Vinicius Junior sealed the victory for Real Madrid, who were unbeaten in their 13 Champions League matches this season. This was the first time they won a European Cup/Champions League final without losing a match during the entire campaign. 


In [ ]:
!rm memory.db*

In [ ]:

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string("memory.db")
agentic_chatbot =create_react_agent(chat_gpt, tools=tools,
                                    checkpointer=memory,
                                    messages_modifier=add_sys_prompt)


#function to call  the agent show result per ueser session (called a thread ID)
from IPython.display import display, Markdown
def chat_with_agent(prompt:str, session_id:str):
    # Ensure the message is formatted correctly with a 'user' role
    response = agentic_chatbot.invoke({"messages":['user',prompt]},
                                      {'configurable':{'thread_id': session_id}})
    display(Markdown(response['messages'][-1].content))


In [ ]:
session_id ='bond001'
prompt ="weather in mumbai?"
chat_with_agent(prompt, session_id)

The current weather in Mumbai is light rain shower with a temperature of 25.9°C (78.6°F). The wind is blowing at a speed of 31.3 km/h (19.5 mph) from the west. The humidity is 87% and the cloud cover is 100%. It feels like 29.0°C (84.1°F) and the windchill is 25.9°C (78.6°F).

In [ ]:
session_id ='bond001'
prompt ="what about dubai"
chat_with_agent(prompt, session_id)

The current weather in Dubai is sunny with a temperature of 36.2°C (97.2°F). The wind is blowing at a speed of 17.3 km/h (10.7 mph) from the west. The humidity is 56% and the cloud cover is 0%. It feels like 46.8°C (116.3°F) and the windchill is 36.2°C (97.2°F).

In [ ]:
session_id ='bond001'
prompt ="which one is hotter?"
chat_with_agent(prompt, session_id)

Based on the current weather conditions, Dubai is hotter than Mumbai. The temperature in Dubai is 36.2°C (97.2°F), while in Mumbai it is 25.9°C (78.6°F).

In [ ]:
from IPython.display import Image , display
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
query = """weather in new york give detail statistics
        """
response = agent.invoke({"messages":['user', query]})
from IPython.display import Markdown, display
display(Markdown(response['messages'][-1].content))

The weather in New York City is partly cloudy with a temperature of 23.5 degrees Celsius (74.3 degrees Fahrenheit). The humidity is 86% and the wind is blowing from the west-southwest at 9.7 kilometers per hour (6 miles per hour). The UV index is 6. 
